In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import s3fs
import six
from pyarrow.filesystem import S3FSWrapper

In [2]:
# import some analysis utilities from https://github.com/englehardt/crawl_utils
import sys
sys.path.append('./crawl_utils/')
import domain_utils as du
import analysis_utils as au

In [3]:
BUCKET = 'openwpm-crawls'
CRAWL_DIR = '2018-09-11_top_20k_stateless'
BUCKET_URI = '%s/%s/visits/%%s' % (BUCKET, CRAWL_DIR)
fs = s3fs.S3FileSystem()

## HTTP Requests

In [ ]:
# Load the data
table_name = 'http_requests'
reqs = pq.ParquetDataset(
    BUCKET_URI % table_name,
    filesystem=fs,
    metadata_nthreads=4
).read_pandas().to_pandas()

### Add some additional columns to help with analysis

In [ ]:
# Add the public suffix + 1 of a bunch of the URL columns
reqs['url_ps1'] = reqs['url'].apply(du.get_ps_plus_1)
reqs['top_ps1'] = reqs['top_level_url'].apply(lambda x: du.get_ps_plus_1(x) if x is not None else None)
reqs['loading_ps1'] = reqs['loading_href'].apply(lambda x: du.get_ps_plus_1(x) if x is not None else None)

In [ ]:
# Parse some info out of the call stack
reqs['stack_scripts'] = reqs['req_call_stack'].apply(au.get_script_urls_from_call_stack_as_set)
reqs['stack_ps1s'] = reqs['stack_scripts'].apply(lambda x: set([du.get_ps_plus_1(y) for y in x]))

In [ ]:
total_sites = reqs['top_level_url'].nunique()

### How many sites is doubleclick.net loaded on?

In [ ]:
reqs[reqs['url_ps1'] == 'doubleclick.net'].top_level_url.nunique() / float(total_sites)

In [ ]:
reqs[reqs['url_ps1'] == 'google-analytics.com'].top_level_url.nunique() / float(total_sites)

### What domains does doubleclick.net load other resources from?

In [ ]:
reqs[
    reqs.req_call_stack != ''
]['req_call_stack'].iloc[0]

In [ ]:
reqs[
    reqs.req_call_stack.apply(lambda x: len(x) > 0)
]['stack_ps1s']

In [ ]:
reqs[
    reqs.stack_ps1s.apply(lambda x: 'doubleclick.net' in x)
].groupby('url_ps1').top_level_url.count().sort_values(ascending=False)

## Javascript Calls

In [ ]:
# Load the data
table_name = 'javascript'
js = pq.ParquetDataset(
    BUCKET_URI % table_name,
    filesystem=fs,
    metadata_nthreads=4
).read_pandas().to_pandas()